In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import cv2 as cv
from IPython.display import Image
import matplotlib.pyplot as plt
from utils import load_images_by_domain, split_images, get_orb_features
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [2]:
# Define paths
img_dir = "../OfficeCaltechDomainAdaptation/images"

# Load images by domain
data_by_domain = load_images_by_domain(
    img_dir=img_dir,
    target_size=(300, 300),  # Standardized size
    method="pad",           # Use padding to maintain aspect ratio
    seed=888                # Seed for reproducibility
)

# Split images: Combine amazon and caltech10 into train/val/test
train_data, val_data, test_data = split_images(
    data_by_domain=data_by_domain,
    train_domains=["amazon", "caltech10"],  # Combine these for training and validation
    test_domains=[],                        # Use part of amazon and caltech10 for testing
    train_split=0.7,                        # 60% for training
    val_split=0.2,                          # 20% for validation
    use_train_for_test=True,                # Use part of train_domains for testing
    test_split=0.1,                         # 20% for testing
    seed=888                                # Seed for reproducibility
)

# Summary of splits
print(f"Train images: {len(train_data['images'])}, Train labels: {len(train_data['labels'])}")
print(f"Validation images: {len(val_data['images'])}, Validation labels: {len(val_data['labels'])}")
print(f"Test images: {len(test_data['images'])}, Test labels: {len(test_data['labels'])}")

Train images: 1456, Train labels: 1456
Validation images: 415, Validation labels: 415
Test images: 210, Test labels: 210


In [3]:
n_iter = 50

nfeatures_arr = uniform(loc=300, scale=400).rvs(n_iter).round().astype(int)    # uniform over [300, 700]
patchSize_arr = uniform(loc=3, scale=9).rvs(n_iter).round().astype(int)        # uniform over [3, 12]
scaleFactor_arr = uniform(loc=1.1, scale=0.4).rvs(n_iter).round(2)             # uniform over [1.1, 1.5]
n_clusters_arr = uniform(loc=100, scale=200).rvs(n_iter).round().astype(int)   # uniform over [100, 300]

In [4]:
train_acc = []
val_acc = []
for idx in range(n_iter):
    X_train, kmeans, transformer = get_orb_features(
        imgs=train_data['images'],
        nfeatures=nfeatures_arr[idx],
        patchSize=patchSize_arr[idx],
        scaleFactor=scaleFactor_arr[idx],
        n_clusters=n_clusters_arr[idx]
    )
    X_val, _, _ = get_orb_features(
        imgs=val_data['images'],
        nfeatures=nfeatures_arr[idx],
        patchSize=patchSize_arr[idx],
        scaleFactor=scaleFactor_arr[idx],
        n_clusters=n_clusters_arr[idx],
        kmeans=kmeans,
        transformer=transformer
    )
    
    model = svm.SVC(decision_function_shape='ovr')
    model.fit(X_train, train_data['labels'])
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    train_acc.append(accuracy_score(y_train_pred, train_data['labels']))
    val_acc.append(accuracy_score(y_val_pred, val_data['labels']))

Getting ORB keypoints...


100%|██████████| 1456/1456 [00:06<00:00, 235.26it/s]


No kmeans was provided, so fitting a new one...
Getting ORB keypoints...


100%|██████████| 415/415 [00:00<00:00, 441.73it/s]


Getting ORB keypoints...


...

100%|██████████| 415/415 [00:01<00:00, 287.45it/s]


Getting ORB keypoints...


100%|██████████| 1456/1456 [00:07<00:00, 202.81it/s]


No kmeans was provided, so fitting a new one...
Getting ORB keypoints...


100%|██████████| 415/415 [00:01<00:00, 327.83it/s]


In [5]:
train_acc = np.array(train_acc)
val_acc = np.array(val_acc)

In [6]:
best_idx = val_acc.argmax()
best_params = {
    'nfeatures':nfeatures_arr[best_idx],
    'patchSize':patchSize_arr[best_idx],
    'scaleFactor':scaleFactor_arr[best_idx],
    'n_clusters':n_clusters_arr[best_idx]
}

In [7]:
best_params

{'nfeatures': 584, 'patchSize': 11, 'scaleFactor': 1.24, 'n_clusters': 268}

In [9]:
train_acc[best_idx], val_acc[best_idx], train_acc.min(), val_acc.min()

(0.8921703296703297,
 0.5734939759036145,
 0.6593406593406593,
 0.40481927710843374)